In [1]:
##imports
import pandas
import numpy 
import warnings
import torch
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
"""This file takes already preprocessed text file and performes Bigram model in it. later on I will construct both model.py
and main.py"""

'This file takes already preprocessed text file and performes Bigram model in it. later on I will construct both model.py\nand main.py'

In [6]:
lines = open(r"D:\projects_with_git\Bigram_model\reddit_convos.txt" , 'r').read().splitlines()

In [10]:
"""making a set that contains all the alphabets"""
char = sorted(list((set(''.join(lines)))))

In [11]:
#encoders and decoders
stoi = {s:i + 1 for i,s in enumerate(char)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [13]:
a = torch.zeros((54 , 54) , dtype = torch.int64)

In [14]:
for line in lines:
    line = ['.'] + list(line) + ['.']
    for ch1 , ch2 in zip(line , line[1:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        a[idx1 , idx2] += 1

In [ ]:
plt.figure(figsize = (100 ,100))
plt.imshow(a , cmap = 'Oranges')
for i in range(54):
    for j in range(54):
        chstr = itos[i] + "-" + itos[j]
        plt.text(j ,i, chstr, ha = "center" , va = "bottom" , color = "gray")
        plt.text(j ,i, a[i , j].item(), ha = "center" , va = "top" , color = "gray")
plt.axis('off');
        

In [66]:
"""Creating all the heatmap values into probability values"""
prob = a[0].float()
prob = prob / sum(prob)
prob

tensor([4.3683e-03, 0.0000e+00, 6.2248e-02, 3.8352e-02, 1.6816e-02, 2.4049e-02,
        1.5113e-02, 1.3692e-02, 1.9176e-02, 4.1217e-02, 2.9393e-01, 1.2424e-02,
        4.3918e-03, 1.9129e-02, 3.5181e-02, 2.6856e-02, 2.2452e-02, 1.6510e-02,
        5.9888e-04, 7.8911e-03, 5.1351e-02, 1.0076e-01, 6.0005e-03, 2.2429e-03,
        5.6189e-02, 2.5834e-04, 4.7441e-02, 1.6440e-04, 5.2373e-03, 2.4777e-03,
        1.4561e-03, 1.8319e-03, 1.0686e-03, 1.1743e-03, 1.4678e-03, 3.4171e-03,
        9.7465e-03, 6.4585e-04, 3.1705e-04, 1.9141e-03, 2.0315e-03, 2.1841e-03,
        2.1724e-03, 1.5148e-03, 4.6971e-05, 3.0296e-03, 3.7224e-03, 5.5308e-03,
        8.6896e-04, 1.4091e-04, 4.0865e-03, 1.2095e-03, 3.8751e-03, 2.3485e-05])

In [75]:
gen = torch.Generator().manual_seed(9968)

In [84]:
for iterations in range(100):
    index = 0
    output = []
    while True:
        prob = a[index].float()
        prob = prob / prob.sum()
        index = index = torch.multinomial(prob , num_samples = 1 , replacement = True , generator = gen).item()
        output.append(itos[index])
        if index == 0:
            break
    print(''.join(output))

I joniny t s vellyou dathe.
Therust lld s y s wecatremen fowats.
Bl ous t st wip totarakin gelfeded y are th thote be pe ffethay g Se axtho woone tio.
Well bentanthele ly d g cey u.
He ay dn be r athine ase f om useretoobest clo in the nidde wopay ongl n ng parevere be ta ththave tsocouste wlt algumaper t or s geale perm Itr eser ave wnt m whe.
Thia shielat er e thevest oit.
I me m It Ls her w dackiFuse mewne averodewoucu tam sttofo gghicanises.
Yoloec w ating me t conthave bury wos yothen he.
It pppperseve ogolmoft let s So Ima If kir p jus bar benanofond in o Id imeal h s t ilparu rs we eveng woug thiliore g ld g b bjuat in s Bust mywhathis itoveverbuy teedds s ryouthelong momont I beererondth he anny youne ouesheas ont mpeat chersomosnge Appat aranningutitimetofly whary.
Th d.
Therf st pl ngoma rso bo b t t s are he ativisor t ambioitelileresop ar atheifo nke butrse ghooiournghe s t s wo leg tit onsliny hano cer tes Nove af s.
J.
I d o u s micomothe s.
I somy.
I ncoete al s latthedu